In [1]:
import yt
import numpy as np
import matplotlib.pyplot as plt
from scipy import constants as C

In [2]:
ds_array = []
for ii in range(10):
    ds_array = np.append( ds_array, 
                         yt.TipsyDataset('run699v2.0'+'%02d'%(ii+1)+'00', dataset_type='tipsy', 
                                         unit_base = {"mass":(2.33e5, "Msun")}) )

yt : [INFO     ] 2019-06-27 00:03:05,034 Parameters: current_time              = 1.0000000000000007
yt : [INFO     ] 2019-06-27 00:03:05,035 Parameters: domain_dimensions         = [2 2 2]
yt : [INFO     ] 2019-06-27 00:03:05,036 Parameters: domain_left_edge          = None
yt : [INFO     ] 2019-06-27 00:03:05,036 Parameters: domain_right_edge         = None
yt : [INFO     ] 2019-06-27 00:03:05,037 Parameters: cosmological_simulation   = 0.0
yt : [INFO     ] 2019-06-27 00:03:05,136 Allocating for 2.639e+06 particles (index particle type 'all')
yt : [INFO     ] 2019-06-27 00:03:05,371 Adding 2.639e+06 particles
yt : [INFO     ] 2019-06-27 00:03:05,716 Identified 1.604e+05 octs
yt : [INFO     ] 2019-06-27 00:03:07,086 Parameters: current_time              = 2.0000000000000013
yt : [INFO     ] 2019-06-27 00:03:07,087 Parameters: domain_dimensions         = [2 2 2]
yt : [INFO     ] 2019-06-27 00:03:07,088 Parameters: domain_left_edge          = None
yt : [INFO     ] 2019-06-27 00:03:07,088

In [7]:
centerD_array = np.zeros(4*3)
centerH_array = np.zeros(4*3)
# The centers
with open('center_dwarf.txt',"r") as f:
    aa = f.readlines()
    f.close()
for nn in range(len(aa)):
    centerD_array[nn] = float(aa[nn].replace('\n', ''))
centerD_array = centerD_array.reshape(-1,3) * yt.units.kpc;

with open('center_host.txt',"r") as f:
    aa = f.readlines()
    f.close()
for nn in range(len(aa)):
    centerH_array[nn] = float(aa[nn].replace('\n', ''))
centerH_array = centerH_array.reshape(-1,3) * yt.units.kpc;

cen_array = np.zeros((10,3))
for ii in range(10):
    ds = ds_array[ii]
    if ii < 4:
        centerHost = centerH_array[ii]
    else:
        density = ds.all_data()[('Gas', 'Density')]
        target  = np.where(density == density.max())[0][0]
        del density
        centerHost = ds.all_data()[('Gas', 'Coordinates')][target].in_units('kpc')
        sphHost0 = ds.sphere(centerHost, (20, 'kpc')) 
        centerHost = sphHost0.quantities.center_of_mass(use_gas=False, use_particles=True, particle_type='Stars')
    cen_array[ii] = centerHost
    sphHost1 = ds.sphere(centerHost, (20, 'kpc'))
    sphHost_4_norm = ds.sphere(centerHost, (5, 'kpc'))
    #计算 host 的角动量，作为其法线方向，再算出与法向量垂直的一个向量，当做画图时的北方
    norm_host  = sphHost_4_norm.quantities.angular_momentum_vector(use_gas=False, use_particles=True, particle_type='Stars')
    north_host = yt.YTArray(yt.ortho_find(norm_host), 'kpc')[1]

/home/protesticon/anaconda3/lib/python3.6/site-packages/yt/fields/derived_field.py:256: VisibleDeprecationWarning: The 'particle_velocity_relative' field has been deprecated in favor of 'relative_particle_velocity'.
  dd = self._function(self, data)


In [15]:
centerH_array

YTArray([[ 1.79359155,  0.12245891,  8.43388181],
         [27.59409232, -1.54855565, 29.43674533],
         [32.3860539 , -1.76999165, 20.50390257],
         [32.32528166, -1.6528167 , 23.41182204]]) kpc

In [5]:
#将遮住其他源的图像保存为 fits 格式，将来用作 PyRAF 的输入
import os
from astropy.io import fits

resolve = 0.1 * yt.units.kpc


def Creat_Obs(nn):
    ds = ds_array[nn]
    centerHost = cen_array[nn]
    sphHost1 = ds.sphere(centerHost, (4, 'kpc'))
    sphHost_4_norm = ds.sphere(centerHost, (4, 'kpc'))
    #计算 host 的角动量，作为其法线方向，再算出与法向量垂直的一个向量，当做画图时的北方
    norm_host = sphHost_4_norm.quantities.angular_momentum_vector(
        use_gas=False, use_particles=True, particle_type='Stars')
    north_host = yt.YTArray(yt.ortho_find(norm_host), 'kpc')[1]
    host_galaxy = ds.disk(centerHost, norm_host, (50, "kpc"), (50, "kpc"))
    nx = np.ceil(50 * yt.units.kpc / resolve).astype('int')
    ny = np.ceil(50 * yt.units.kpc / resolve).astype('int')
    obs = yt.YTArray(np.zeros((ny, nx)), 'Msun')
    for ii in range(nx):
        print(ii)
        for jj in range(ny):
            booleanx = (host_galaxy['Stars', 'relative_particle_position_x']>(ii*resolve-0.5*nx*resolve))\
                & (host_galaxy['Stars', 'relative_particle_position_x']<=((ii+1)*resolve-0.5*nx*resolve))
            booleany = (host_galaxy['Stars', 'relative_particle_position_z']>(jj*resolve-0.5*ny*resolve))\
                & (host_galaxy['Stars', 'relative_particle_position_z']<=((jj+1)*resolve-0.5*ny*resolve))
            obs[jj, ii] = (host_galaxy['Stars', 'Mass'][booleanx
                                                        & booleany]).sum()
    if os.path.exists('fits/699_%02d' % (nn + 1) + '_1.fits'):
        os.remove('fits/699_%02d' % (nn + 1) + '_1.fits')
    image_input = obs.copy()
    grey = fits.PrimaryHDU(image_input)
    greyHDU = fits.HDUList([grey])
    greyHDU.writeto('fits/699_%02d' % (nn + 1) + '_1.fits')
    del image_input
    
Creat_Obs(5)

/home/protesticon/anaconda3/lib/python3.6/site-packages/yt/fields/derived_field.py:256: VisibleDeprecationWarning: The 'particle_velocity_relative' field has been deprecated in favor of 'relative_particle_velocity'.
  dd = self._function(self, data)


0


/home/protesticon/anaconda3/lib/python3.6/site-packages/yt/fields/derived_field.py:256: VisibleDeprecationWarning: The 'particle_position_relative' field has been deprecated in favor of 'relative_particle_position'.
  dd = self._function(self, data)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [10]:
t_height = 4
r_3 = 18.5

In [12]:
for ii in range(10):
    ds = ds_array[ii]
    centerHost = cen_array[ii]
    sphHost1 = ds.sphere(centerHost, (5, 'kpc'))
    sphHost_4_norm = ds.sphere(centerHost, (5, 'kpc'))
    #计算 host 的角动量，作为其法线方向，再算出与法向量垂直的一个向量，当做画图时的北方
    norm_host = sphHost_4_norm.quantities.angular_momentum_vector(
        use_gas=False, use_particles=True, particle_type='Stars')
    north_host = yt.YTArray(yt.ortho_find(norm_host), 'kpc')[1]
    pp_all_star = yt.OffAxisProjectionPlot(ds,
                                           norm_host,
                                           ('deposit', 'Stars_density'),
                                           center=centerHost,
                                           width=(4*r_3, 'kpc'),
                                           depth=(t_height,  'kpc'),
                                           north_vector=north_host,
                                           method='integrate',
                                           fontsize=40)

    # Ensure the colorbar limits match for all plots
    pp_all_star.set_unit(('deposit', 'Stars_density'), 'Msun/pc**2')
    pp_all_star.set_cmap(('deposit', 'Stars_density'), 'jet')
    pos_of_annotate = yt.YTArray([-25, 20], input_units='kpc')
    pp_all_star.annotate_clear()
    pp_all_star.annotate_text(pos_of_annotate,
                              '%2d' % (ii + 1) + ' Gyr',
                              coord_system='plot',
                              text_args={'color': 'red'})
    pp_all_star.set_zlim(('deposit', 'Stars_density'), 1e-4, 1e5)

    pp_all_star.save('figs/ProjectionPlots_AM/699_' + '%02d' % (ii + 1) + '00_all_star0.png',
                     mpl_kwargs={'dpi': 400})

yt : [INFO     ] 2019-06-27 00:27:47,482 xlim = -37.000000 37.000000
yt : [INFO     ] 2019-06-27 00:27:47,483 ylim = -37.000000 37.000000
yt : [INFO     ] 2019-06-27 00:27:47,486 zlim = -2.000000 2.000000
yt : [INFO     ] 2019-06-27 00:27:47,494 Making a fixed resolution buffer of (('deposit', 'Stars_density')) 800 by 800
yt : [INFO     ] 2019-06-27 00:28:09,939 Saving plot figs/699_0100_all_star0.png
/home/protesticon/anaconda3/lib/python3.6/site-packages/yt/fields/derived_field.py:256: VisibleDeprecationWarning: The 'particle_velocity_relative' field has been deprecated in favor of 'relative_particle_velocity'.
  dd = self._function(self, data)
yt : [INFO     ] 2019-06-27 00:28:13,960 xlim = -37.000000 37.000000
yt : [INFO     ] 2019-06-27 00:28:13,960 ylim = -37.000000 37.000000
yt : [INFO     ] 2019-06-27 00:28:13,962 zlim = -2.000000 2.000000
yt : [INFO     ] 2019-06-27 00:28:13,965 Making a fixed resolution buffer of (('deposit', 'Stars_density')) 800 by 800
yt : [INFO     ] 2019

In [13]:
for ii in range(10):
    ds = ds_array[ii]
    centerHost = cen_array[ii]
    sphHost1 = ds.sphere(centerHost, (5, 'kpc'))
    sphHost_4_norm = ds.sphere(centerHost, (5, 'kpc'))
    #计算 host 的角动量，作为其法线方向，再算出与法向量垂直的一个向量，当做画图时的北方
    norm_host = sphHost_4_norm.quantities.angular_momentum_vector(
        use_gas=False, use_particles=True, particle_type='Stars')
    north_host = yt.YTArray(yt.ortho_find(norm_host), 'kpc')[1]
    pp_all_star = yt.OffAxisProjectionPlot(ds,
                                           north_host,
                                           ('deposit', 'Stars_density'),
                                           center=centerHost,
                                           width=(4*r_3, 'kpc'),
                                           #depth=(4,  'kpc'),
                                           north_vector=norm_host,
                                           method='integrate',
                                           fontsize=40)

    # Ensure the colorbar limits match for all plots
    pp_all_star.set_unit(('deposit', 'Stars_density'), 'Msun/pc**2')
    pp_all_star.set_cmap(('deposit', 'Stars_density'), 'jet')
    pos_of_annotate = yt.YTArray([-25, 20], input_units='kpc')
    pp_all_star.annotate_clear()
    pp_all_star.annotate_text(pos_of_annotate,
                              '%2d' % (ii + 1) + ' Gyr',
                              coord_system='plot',
                              text_args={'color': 'red'})
    pp_all_star.annotate_line([0, 0.5 - t_height/(8*r_3)], [1, 0.5 - t_height/(8*r_3)], coord_system='axis',
                    plot_args={'color':'black', 'linestyle':'--'})
    pp_all_star.annotate_line([0, 0.5 + t_height/(8*r_3)], [1, 0.5 + t_height/(8*r_3)], coord_system='axis',
                    plot_args={'color':'black', 'linestyle':'--'})
    pp_all_star.set_zlim(('deposit', 'Stars_density'), 1e-4, 1e5)

    pp_all_star.save('figs/ProjectionPlots_AM/699_' + '%02d' % (ii + 1) + '00_all_star1.png',
                     mpl_kwargs={'dpi': 400})

/home/protesticon/anaconda3/lib/python3.6/site-packages/yt/fields/derived_field.py:256: VisibleDeprecationWarning: The 'particle_velocity_relative' field has been deprecated in favor of 'relative_particle_velocity'.
  dd = self._function(self, data)
yt : [INFO     ] 2019-06-27 00:33:50,401 xlim = -37.000000 37.000000
yt : [INFO     ] 2019-06-27 00:33:50,401 ylim = -37.000000 37.000000
yt : [INFO     ] 2019-06-27 00:33:50,404 zlim = -806.995375 806.995375
yt : [INFO     ] 2019-06-27 00:33:50,406 Making a fixed resolution buffer of (('deposit', 'Stars_density')) 800 by 800
yt : [INFO     ] 2019-06-27 00:34:12,659 Saving plot figs/699_0100_all_star1.png
/home/protesticon/anaconda3/lib/python3.6/site-packages/yt/fields/derived_field.py:256: VisibleDeprecationWarning: The 'particle_velocity_relative' field has been deprecated in favor of 'relative_particle_velocity'.
  dd = self._function(self, data)
yt : [INFO     ] 2019-06-27 00:34:15,924 xlim = -37.000000 37.000000
yt : [INFO     ] 2019-